In [1]:
import sys
import os

sys.path.append(os.path.dirname('/home/insane/u/APalysis/analysis'))

import keract
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
from nltk.corpus import wordnet as wn
from tensorflow import keras as K
from tqdm import tqdm
from analysis import metrics
import utils

# check if using GPU
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num CPUs Available: ", len(tf.config.experimental.list_physical_devices('CPU')))

2024-03-19 16:34:28.848190: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-19 16:34:28.848213: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-19 16:34:28.849059: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-19 16:34:28.853725: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-19 16:34:29.468792: W tensorflow/compiler/tf2

Num GPUs Available:  1
Num CPUs Available:  1


2024-03-19 16:34:31.281015: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-19 16:34:31.296171: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-19 16:34:31.296388: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [4]:
model = tf.keras.applications.inception_v3.InceptionV3( weights='imagenet')
model.compile(loss="categorical_crossentropy", optimizer="adam")

ds, info = tfds.load(
    'imagenet2012',
    shuffle_files=False,
    with_info=True,
    as_supervised=True,
    batch_size=None,
    data_dir='/run/media/insane/Games/Tensorflow/tensorflow_datasets'
)
labels = list(map(lambda l: wn.synset_from_pos_and_offset(
        l[0], int(l[1:])).name(), info.features['label'].names))
ds = ds['train']


inception_input_shape = tf.keras.applications.inception_v3.InceptionV3().input.shape[1:3].as_list()
@tf.function
def preprocess(x, y):
    x = tf.image.resize(x, inception_input_shape, method=tf.image.ResizeMethod.BILINEAR)
    x = tf.keras.applications.inception_v3.preprocess_input(x)
    return x, y

def preprocess_inv(x, y):
    x = ((x / 2 + 0.5) * 255).astype(np.uint8).squeeze()
    return x, y

In [ ]:
# draw barchart on how many images per class is there in imagenet
label_counts = np.zeros(len(labels))
for x, y in tqdm(ds):
    label_counts[y] += 1
total = np.sum(label_counts)
fig = plt.figure(figsize=(20, 10))
plt.bar(labels, label_counts)
plt.xticks(rotation=90)
plt.show()

In [5]:
def print_dict_numpy(d):
    '''Prints a dictionary with numpy arrays recursively. Numpy arrays are printed as their shape and dtype.'''
    if not isinstance(d, dict):
        print(d)
        return
    for k, v in d.items():
        if isinstance(v, dict):
            print(k, ":", end=" ")
            print_dict_numpy(v)
        elif isinstance(v, np.ndarray):
            print(k, ":", v.shape, v.dtype)
        else:
            print(k, ":", v)

In [6]:
BATCH_SIZE = 1
summary_fn_image = metrics.summary_fn_image_l2
summary_fn_dense = metrics.summary_fn_dense_identity

layers = list(map(lambda layer: layer.name, filter(lambda l: isinstance(l, (
    # K.layers.InputLayer,
    K.layers.Conv2D,
    # K.layers.Dense,
    # K.layers.Flatten,
    K.layers.Concatenate,
)), model.layers)))

labels_so_far = np.zeros(len(labels), dtype=np.int32)
MAX_COUNT = 100
layers_to_save = ['mixed10', 'mixed9', 'mixed7']
# layers_to_save = layers

for class_label in tqdm(range(301, 1000)):
    # __datasetImgs = [[] for _ in range(len(labels))]
    # __activations = [[] for _ in range(len(labels))]
    __activationsSummary = {k: [] for k in layers_to_save}
    # __datasetLabels = []
    dataset = ds.map(preprocess).batch(BATCH_SIZE).shuffle(1000)
    # printed = False
    for i, (img, label) in enumerate(dataset):
        if label.numpy().item() != class_label:
            continue

        # if not printed:
        #     print("Class :", label.numpy().item())
        #     plt.imshow(img[0])
        #     plt.show()
        #     printed = True

        # Get activations
        activation = keract.get_activations(
            model, img, layer_names=layers_to_save, nodes_to_evaluate=None, output_format='simple', nested=False, auto_compile=True)

        activationSummary = {}
        for layer, v in activation.items():
            if layer not in layers_to_save: continue
            if len(v[0].shape) == 1:
                # dense layer
                activationSummary[layer] = summary_fn_dense(v)
            elif len(v[0].shape) == 3:
                # Image layer
                activationSummary[layer] = summary_fn_image(v)
                
        # labels_so_far[label.numpy()] += 1
        # __datasetLabels += label.numpy().tolist()
        
        for layer, summary in activationSummary.items():
            if layer not in layers_to_save: continue
            __activationsSummary[layer] += summary.tolist()
            
        if layers_to_save[0] in __activationsSummary and len(__activationsSummary[layers_to_save[0]]) >= MAX_COUNT:
            break
            
            
    # datasetImgs = [j for i in __datasetImgs for j in i]
    # activations = [j for i in __activations for j in i]
    activationsSummary = {k: np.array(v) for k, v in __activationsSummary.items()}
    # datasetLabels = np.array(__datasetLabels)
    
    # save the activation summary
    np.savez_compressed(f'./activation_summary/activations_{class_label}.npz', **activationsSummary)
    
# Get the prediction with argmax
# predictions = []
# for i in range(len(activations)):
#     predictions.append(np.argmax(activations[i][layers[-1]][0]).item())

print_dict_numpy(activationsSummary)
# print(datasetLabels.shape)

  0%|          | 0/699 [00:00<?, ?it/s]

2024-03-19 16:35:13.117759: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8902
2024-03-19 16:35:13.388780: W external/local_xla/xla/service/gpu/llvm_gpu_backend/gpu_backend_lib.cc:504] Can't find libdevice directory ${CUDA_DIR}/nvvm/libdevice. This may result in compilation or runtime failures, if the program we try to run uses routines from libdevice.
Searched for CUDA in the following directories:
  ./cuda_sdk_lib
  /usr/local/cuda-12.2
  /usr/local/cuda
  /home/insane/.cache/pypoetry/virtualenvs/apalysis-5kmHI_dQ-py3.10/lib/python3.10/site-packages/tensorflow/python/platform/../../../nvidia/cuda_nvcc
  /home/insane/.cache/pypoetry/virtualenvs/apalysis-5kmHI_dQ-py3.10/lib/python3.10/site-packages/tensorflow/python/platform/../../../../nvidia/cuda_nvcc
  .
You can choose the search directory by setting xla_gpu_cuda_data_dir in HloModule's DebugOptions.  For most apps, setting the environment variable XLA_FLAGS=--xla_gpu_cuda_data_dir=/path/to/cuda